<a href="https://colab.research.google.com/github/PaulinaPacyna/SeoAI/blob/master/GTP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.


In [2]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!nvidia-smi

Tue Feb  1 20:42:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
model_name = "124M"
gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 483Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 2.95Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 738Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:16, 31.0Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 715Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.81Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 3.73Mit/s]


If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [5]:
for file_name in ["stack.txt", "amazon_descriptions.txt"]:
  gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [7]:
# sess = gpt2.start_tf_sess()
for file_name in [ "amazon_descriptions.txt"]:
  gpt2.finetune(sess,
                dataset=file_name,
                model_name="124M",
                steps=1000,
                restore_from='latest',
                run_name='s_a',
                print_every=100,
                sample_every=200,
                save_every=500,
                overwrite=True,
                reuse=True
                )

Loading checkpoint checkpoint/s_a/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/s_a/model-1000
Loading dataset...


100%|██████████| 1/1 [00:57<00:00, 57.35s/it]


dataset has 11925811 tokens
Training...
Saving checkpoint/s_a/model-1000
Saving checkpoint/s_a/model-1000
======== SAMPLE 1 ========
 dog, so that you have a reasonable chance of making it through to the next stage). i am extremely unsure how best to start the process. here is the list of things you can do: take your puppy in for her examination. she is likely already gotten on well with the medical staff, as she was expected to, but she is definitely got a fair bit of "overreacting" going on. for most children i guess, her anxiety can be a bit of a distraction. this is a good time to take it slow and work on positive reinforcement. you can do a small reward training as well - there is a bit of that on here too :). you need to get it to a point where you start out with a high reward. this may take some time, but once that is working at the level it is, then this approach may be working for you, too. 
i do not have a dog, i just have two, the most interesting i could find was this. i do

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [8]:
gpt2.copy_checkpoint_to_gdrive(run_name='s_a')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [8]:
gpt2.copy_checkpoint_from_gdrive(run_name='s_a')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [4]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='s_a')

Loading checkpoint checkpoint/s_a/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/s_a/model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [5]:
gpt2.generate(sess, 
              prefix='is salmon good for a dog?',
              include_prefix=False,
              temperature = 0.6,
              top_p=0.9,
              truncate='\n',
              nsamples=5,
              batch_size=5,  
              run_name='s_a')

 yes, you can add a small amount to your dog is diet to help with the digestion process. it is best to start with small amounts, and increase as you see the need for them. a small amount of salmon is not too bad for a dog. 
 i would not worry too much about it. the best thing about this is that it is not a big deal, but you should be aware of the risks. the reason why i would be worried about this is that it is a good thing that there is a water quality test done to determine if there is any bacteria present in the fish. it is also possible that there are some other bacteria present in the water that are not present in the fish, and thus there is some risk of some bacteria in the water. if this is the case, then i would not worry about it. 
 this is not a good idea, as it could lead to a dog who is not eating enough and you will end up with a dog who is not eating enough. 
 i am not sure what the fish are that are good for fish. what fish are you talking about? 
is salmon good for a do

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.